# Test CNN model

## Goals

Current file aims to set up the CNN model for the example

In [1]:
import codecs
import os
import sys
import zipfile
import pathlib

import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow_datasets as tfds

In [2]:
(train_ds, test_ds), metadata = tfds.load('colorectal_histology',split=['train[:80%]','train[80%:]'],with_info=True,
        as_supervised=True)

In [3]:
class CNN(tf.keras.Sequential):
    def __init__(self):
        super(CNN, self).__init__(layers=[
            tf.keras.layers.Rescaling(1./255, input_shape=(150,150,3)),
            tf.keras.layers.Conv2D(32, 5, activation='relu'),
            tf.keras.layers.MaxPooling2D(),
            tf.keras.layers.Conv2D(64, 5, activation='relu'),
            tf.keras.layers.MaxPooling2D(),
            tf.keras.layers.Conv2D(128, 5, activation='relu'),
            tf.keras.layers.MaxPooling2D(),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(128, activation='relu'),
            tf.keras.layers.Dense(8, activation= 'softmax')
        ])

In [13]:
train_ds

<_PrefetchDataset element_spec=(TensorSpec(shape=(150, 150, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [4]:
model = CNN()

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

criterion = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)

In [5]:
model.compile(optimizer=optimizer, loss=criterion, metrics=['accuracy'])

In [6]:
model.summary()

Model: "cnn"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 150, 150, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 146, 146, 32)      2432      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 73, 73, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 69, 69, 64)        51264     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 34, 34, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 30, 30, 128)       204928  

In [7]:
15*15*32

7200

In [10]:
# model.fit(
#   train_ds.batch(32),
#   epochs=1
# )

model.fit(
  train_ds,
  batch_size=32,
  epochs=1
)

TypeError: in user code:

    File "c:\miniconda\lib\site-packages\keras\engine\training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "c:\miniconda\lib\site-packages\keras\engine\training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\miniconda\lib\site-packages\keras\engine\training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "c:\miniconda\lib\site-packages\keras\engine\training.py", line 1055, in train_step
        return self.compute_metrics(x, y, y_pred, sample_weight)
    File "c:\miniconda\lib\site-packages\keras\engine\training.py", line 1149, in compute_metrics
        self.compiled_metrics.update_state(y, y_pred, sample_weight)
    File "c:\miniconda\lib\site-packages\keras\engine\compile_utils.py", line 605, in update_state
        metric_obj.update_state(y_t, y_p, sample_weight=mask)
    File "c:\miniconda\lib\site-pac

In [9]:
model.evaluate(test_ds.batch(32), verbose=2)

32/32 - 7s - loss: 0.9559 - accuracy: 0.5590 - 7s/epoch - 213ms/step


[0.9558648467063904, 0.5590000152587891]